## Azure AI Foundry 에이전트

MAF(Microsoft Agent Framework)는 Azure AI Foundry Agent 서비스를 사용하는 에이전트를 만드는 것도 지원합니다. 서비스에서 자체 제공하는 대화 스레드를 활용해서 영구적인 서비스-기반의 에이전트 인스턴스를 만들 수 있습니다.

#### 환경 변수  

Azure AI Foundry Agen를 사용하기 전에 다음 환경 변수를 설정해야 합니다. 또는, 코드에서 직접 지정해도 됩니다.   
다만, [0.Prerequisite.ipynb - 사전 준비 사항 및 환경 설정](0.Prerequisite.ipynb) 단계에서 이미 .env 파일에 설정했다면, 여기서 별도로 설정할 필요는 없습니다.

> ``` bash
> export AZURE_AI_PROJECT_ENDPOINT="https://<your-project>.services.ai.azure.com/api/projects/<project-id>"
> export AZURE_AI_MODEL_DEPLOYMENT_NAME="gpt-4o-mini"
>```

#### 설치

Agent Framework의 Azure AI 패키지를 프로젝트에 추가합니다

In [ ]:
# agent-framework 설치. 2026.01.30 릴리즈된 베타 버전 (Preview)
%pip install agent-framework==1.0.0b260130   

### Azure AI Foundry 에이전트

#### 기본 에이전트 생성하기

가장 빠르게 에이전트를 생성하는 방법은 AzureAIAgentClient를 사용하는 것입니다. 환경 변수를 사용하여 에이전트를 생성할 수 있습니다. 단, 이 에이전트는 영구적이지 않으며, 세션이 종료되면 사라집니다.

In [ ]:
import asyncio
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

async def main():
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            credential=credential,
            agent_name="HelperAgent"
        ).as_agent(
            instructions="You are a helpful assistant.",
            store=True
        ) as agent,
    ):
        result = await agent.run("Hello!")
        print(result.text)

await main()

### Foundry에 유지되는 에이전트 생성 및 관리 (Classic Portal)

에이전트 수명주기를 더 제어하기 위해서는, Azure AI Projects 클라이언트를 사용하여 Foundry에 유지되는 에이전트를 생성합니다. 다음의 코드는 현재 Fodunry 프로젝트에 존재하는 에이전트 목록을 나열하고, 지정한 이름의 에이전트가 없으면 새로 생성하는 예제입니다. 
우선, 이번 실습에서는 AgentsClient를 사용하여 Persistent Agent를 생성합니다.    

이 경우, Agent는 **신규 Foundry 포탈이 아닌, Classic 포탈에 생성됨에 유의**합니다.

In [ ]:
# agent_framework 패키지의 실제 구조 확인
import agent_framework
from agent_framework.azure import AzureOpenAIChatClient

print("=== agent_framework 모듈 확인 ===")
print(f"agent_framework 버전: {agent_framework.__version__ if hasattr(agent_framework, '__version__') else 'N/A'}")
print(f"\nagent_framework 모듈 내용:")
print([x for x in dir(agent_framework) if not x.startswith('_')])

In [ ]:
import os
from agent_framework.azure import AzureAIAgentClient
from agent_framework import ChatAgent
from azure.identity import AzureCliCredential
from azure.ai.agents import AgentsClient

# 이 Agent는 신규 Foundry 포탈이 아닌, Classic 포탈에 생성됩니다.
credential = AzureCliCredential()
agents_client = AgentsClient(
    endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    credential=credential
)

#새로 생성할 에이전트 이름
agent_name="PersistentAgent"

# 기존 에이전트 목록 조회
print("=== 기존 에이전트 목록 ===")
for agent in agents_client.list_agents():
    print(f"Agent ID: {agent.id}, Name: {agent.name}")

# 기존 Agent가 없으면 새로운 Agent 생성
agent = None
for existing in agents_client.list_agents():
    if existing.name == agent_name:
        agent = existing
        break

if(agent):
    print(f"에이전트 '{agent_name}'(이)가 이미 존재합니다. 새로 생성하지 않습니다.")
else:
    print(f"에이전트 '{agent_name}'(이)가 존재하지 않습니다. 새로 생성합니다.")
    agent = agents_client.create_agent(
        model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"],
        name=agent_name,
        instructions="You are a helpful assistant who speak in korean"
    )

print(f"에이전트 ID: {agent.id}")
print(f"에이전트 이름: {agent.name}")
print(f"모델: {agent.model}")
print(f"instruction : {agent.instructions}")

# 기존(또는 생성된) 에이전트를 사용하여 ChatAgent 구성
chat_agent = ChatAgent(
    chat_client=AzureAIAgentClient(
        credential=credential,
        agent_id=agent.id
    )
)
result = await chat_agent.run("오늘도 좋은 하루!")
print(result.text)

agent_id=agent.id
# 정리 (필요시 주석 해제)
# agents_client.delete_agent(agent.id)
# print(f"\n에이전트 삭제 완료: {agent.id}")

#### 기존 Azure AI Foundry 에이전트 사용하기 (Classic Portal)

만일 Azure AI Foundry 프로젝트에 이미 에이전트가 존재한다면, 그 에이전트의 ID를 지정하여 기존 에이전트의 인스턴스를 만들고 사용할 수도 있습니다. 다음 예제에서는 이전에 생성된 에이전트의 ID를 사용하여 에이전트를 불러오고, 질문을 던지는 방법을 보여줍니다.

In [ ]:
import asyncio
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

agent_id="<이전의 agent id>"  # 이전에 생성된 에이전트를 여기에 할당
agent_id="asst_dwy8NMHcX8g0Id5w4KQssTZM"
async def main():
    async with (
        AzureCliCredential() as credential,
        ChatAgent(
            chat_client=AzureAIAgentClient(
                credential=credential,
                agent_id=agent_id
            )
        ) as agent,
    ):
        
        result = await agent.run("한국은 어디에 있나요?")
        print(result.text)

await main()

Azure AI Foundry 에이전트는 그 밖의 일반적인 에이전트 기능, 다음과 같은 기능들도 모두 지원합니다.

- Function Tools
- Code Interpreter
- Streaming Responses

### Foundry에 유지되는 에이전트 생성 및 관리 (New Portal)

아직은 Preview인 신규 포탈을 사용하여 에이전트 수명주기를 제어하려면, AIProjectClient를 사용하여 Foundry에 유지되는 에이전트를 생성합니다. 다음의 코드는 현재 Fodunry 프로젝트에 존재하는 에이전트 목록을 나열하고, 지정한 이름의 에이전트가 없으면 새로 생성하는 예제입니다. 

이 경우, Agent는 **신규 Foundry 포탈에 생성**됩니다. 이 에이전트는 신규 포탈에서 지원하는 완전한 Observability 기능도 활용할 수 있습니다.

In [ ]:
import asyncio
import os
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import AzureCliCredential
from azure.ai.projects.models import AgentReference, PromptAgentDefinition

import os
from agent_framework.azure import AzureAIAgentClient
from agent_framework import ChatAgent
from azure.identity import DefaultAzureCredential
from azure.ai.agents import AgentsClient

# 새로 생성할 에이전트 이름
AGENT_NAME = "BeerAgent"

async with (
    AIProjectClient(
        endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
        credential=DefaultAzureCredential()
    ) as project_client,
):
    # 기존 Agent 찾기
    agent = None
    async for existing in project_client.agents.list():
        if existing.name == AGENT_NAME:
            agent = existing
            break
    
    ## 기존 에이전트가 없으면 새로운 에이전트 생성
    if(agent):
        print(f"에이전트 '{AGENT_NAME}'(이)가 이미 존재합니다. 새로 생성하지 않습니다.")
    else:
        # 에이전트 생성
        print(f"에이전트 '{AGENT_NAME}'(이)가 존재하지 않습니다. 새로 생성합니다.")
        definition = PromptAgentDefinition(
            model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"],
            instructions="You are a Beer and Ale knowledgeable assistant who speaks in Korean"
        )                
        agent = await project_client.agents.create(
            name=AGENT_NAME,
            definition=definition
        )
        # 생성된 에이전트 정보 출력
        print(f"Created Agent Name: {agent.name}")  

    # OpenAI client 가져오기
    openai_client = project_client.get_openai_client()
    
    # Agent 정보를 본문에 추가하여 Response 생성 요청
    response = await openai_client.responses.create(
        input=[{"role": "user", "content": "겨울에 어울리는 맥주는?"}],
        extra_body={"agent": AgentReference(name=agent.name).as_dict()}
    )
    
    print(f"Agent response: {response.output_text}")

![MAF-agent-newportal.png](./images/MAF-agent-newportal.png)